# I. pre-requis

In [ ]:
# 1)  import package
import pandas as pd
import numpy as np
import requests
from requests.auth import HTTPBasicAuth # for RTB House

In [7]:
# 2) Define neccessary functions

# 1) read_googlesheet()
def read_googlesheet(sheet_id, worksheet_name):

  # Import packages
  import gspread
  from google.oauth2.service_account import Credentials
  import pandas as pd

  # Authenticate with the service account, specifying the required scopes

  # credential_info
  credentials_info = json.loads(os.environ.get("GSPREAD_API_CREDENTIALS_INFO"))

  # 1. Authenticate with the service account, specifying the required scopes
  scopes = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']
  credentials = Credentials.from_service_account_info(credentials_info, scopes=scopes)

  # 2. Initialize the gspread client with the service account credentials
  gc = gspread.authorize(credentials)
  sh = gc.open_by_key(sheet_id) #Establishing connection to your sheet
  worksheet = sh.worksheet(worksheet_name) #Specifying the worksheet used

  # 3. get data
  rows = worksheet.get_all_values()
  df = pd.DataFrame(rows[1:], columns=rows[0])

  # 4. Return table
  return df
  display(df)

In [6]:
def upload_googlesheet(df, sheet_id, worksheet_name):

  # Import packages
  import gspread
  from google.oauth2.service_account import Credentials
  import pandas as pd

  # Authenticate with the service account, specifying the required scopes

  # credential_info
  credentials_info = json.loads(os.environ.get("GSPREAD_API_CREDENTIALS_INFO"))

  # 1. Authenticate with the service account, specifying the required scopes
  scopes = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']
  credentials = Credentials.from_service_account_info(credentials_info, scopes=scopes)

  # 2. Initialize the gspread client with the service account credentials
  gc = gspread.authorize(credentials)
  sh = gc.open_by_key(sheet_id) #Establishing connection to your sheet
  worksheet = sh.worksheet(worksheet_name) #Specifying the worksheet used

  # 3. Clear table
  worksheet.clear()

  # 4. Uploading data
  data_values = [df.columns.values.tolist()] + df.values.tolist()
  worksheet.update(data_values)

# II. fetch Google ads

In [ ]:
# Load the config into the client
from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.config import load_from_dict
import os
import json

GOOGLE_ADS_API_KEY =  json.loads(os.environ.get("GOOGLE_ADS_API_KEY")) 
client = GoogleAdsClient.load_from_dict(GOOGLE_ADS_API_KEY)


# Get the GoogleAdsService client
ga_service = client.get_service("GoogleAdsService")

# === Step 2 : Query Immobilier data test ===

# Specify the Google Ads account ID (Customer ID) you want to query, without dashes
customer_id = "7054858844" #Leboncoin - Immobilier

# Define a GAQL query to get campaign performance metrics
query = (
f"""
SELECT
  segments.date,
  campaign.name,
  campaign.id,
  ad_group.name,
  ad_group_ad.ad.id,
  metrics.impressions,
  metrics.clicks,
  metrics.cost_micros,
  metrics.conversions
FROM
  ad_group_ad
WHERE
  segments.date DURING LAST_30_DAYS
AND
  campaign.name LIKE "%_perf_%"
ORDER BY
  segments.date ASC
"""
)

# Issue a search request
response = ga_service.search(customer_id=customer_id, query=query)



# === Step 3 : Convert to table ===

import pandas as pd

rows = []

for row in response:
  rows.append({
      "Date" : row.segments.date,
      "Campaign" : row.campaign.name,
      "Campaign ID" : row.campaign.id,
      "Ad Group" : row.ad_group.name,
      "Ad Name" : row.ad_group_ad.ad.id,
      "Vertical" : "Real Estate",
      "Ad platform" : "Google",
      "Goal" : "Perf",
      "Category" : "-",
      "Sub-category" : "-",
      "Impressions" : row.metrics.impressions,
      "Clicks" : row.metrics.clicks,
      "Cost" : row.metrics.cost_micros/1_000_000,
      "Replies" : row.metrics.conversions
  })

df = pd.DataFrame(rows)

display(df.head())

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [ ]:
# Update data to googlesheet

upload_googlesheet(df, "1mpolpr22uDXC_mQx9oEi72tt39GjroByPprB-HLtpY4", "Sheet5")